### Setup

Welcome to the OSS Insight setup guide.

This guide will help you setup your environment to use OSS Insight.

#### Prerequisites

Before you begin, you need to have the following installed:

- TiDB Cluster - Database to store the data
  MyCLI - Connect to the TiDB Cluster
- Node.js 18.x and above - API server runtime
- PNPM - Package manager for Node.js
- Python 3.x and pip - For jupyter notebook

##### 1. Startup TiDB Cluster

First of all, you need to startup a TiDB cluster. You can startup a local cluster for testing by [TiUP Playground](https://docs.pingcap.com/tidb/dev/tiup-playground#tiup-playground-overview). But in this time, we recommend that you start a **serverless tier** cluster using TiDB Cloud, please click [here](https://docs.pingcap.com/tidbcloud/tidb-cloud-quickstart#step-1-create-a-tidb-cluster) to learn how to startup a serverless tier cluster.

If you already have a TiDB cluster that you can connect to, you can ignore this step.

##### 2. Prepare database connection information

<center>
  <img align="middle" width="800" alt="Serverless Tier Cluster Manage Interface" src="https://user-images.githubusercontent.com/5086433/204476069-0ddbdf6f-419c-4291-b929-ccfbd2f5ea5f.png">
</center>

You can get the following information on the interface.

##### 3. Prepare your GitHub access token

You need to prepare a personal access token to allow the application to access the data through the GitHub API.

> **Note**
>
> If you are viewing this document from GitHub codespace, you can skip this step because `GITHUB_TOKEN` is set by default in the codespace environment.
> 

Creating a persYou can learn how to generate one by reading: [Creating a personal access token](https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/creating-a-personal-access-token). Or just click this [link](https://github.com/settings/personal-access-tokens/new) to generate your personal access token quickly.

<center>
  <img align="middle" width="800" alt="Create a new GitHub personal accesst token" src="https://user-images.githubusercontent.com/5086433/204564273-93cccbe4-d10a-4d1b-a9d1-112a1144712a.png">
</center>

##### 4. Setup the environment variables

Execute the following python code, which is written to guide you in setting the required environment variables:

In [31]:
import getpass
import os

api_server_dot_env="./packages/api-server/.env"
if os.path.exists(api_server_dot_env): 
    con = input(api_server_dot_env + " file already exists, do you want to overwrite it? (y/n)")
    exit()

# Config GitHub personal access token.
github_token = ""
if os.getenv('GITHUB_TOKEN1') is None:
    github_token = getpass.getpass(prompt='Enter your personal access token of GitHub: ')
else:
    github_token = os.getenv('GITHUB_TOKEN')

# Config database connection.
db_endpoint = input("The endpoint of TiDB cluster: ")
db_port = input("The port of TiDB cluster [4000]: ")

if db_port == "":
    db_port = 4000
else:
    db_port = int(db_port)

db_username = input("The username of TiDB cluster: ")
db_password = getpass.getpass(prompt='Enter the password of TiDB cluster: ')
db_name = "ossinsight"

db_enable_ssl = input("Enable ssl connection to the TiDB cluster? (y/n)")
db_ssl_config=""
if db_enable_ssl == "y":
    db_ssl_config='&{"minVersion":"TLSv1.2"}'

# Setup system environment variables.
os.environ['GITHUB_TOKEN'] = github_token
os.environ['DB_ENDPOINT'] = db_endpoint
os.environ['DB_PORT'] = str(db_port)
os.environ['DB_USERNAME'] = db_username
os.environ['DB_PASSWORD'] = db_password
os.environ['DB_NAME'] = db_name
os.environ['DB_ENABLE_SSL'] = db_enable_ssl

# Write to ./packages/api-server/.env file.
with open(api_server_dot_env, "w") as file:
    file.write(
        "DATABASE_URL=mysql://{}:{}@{}:{}/{}?connectionLimit=100&queueLimit=10000{}\n".format(
            db_username, db_password, db_endpoint, db_port, db_name, db_ssl_config
        )
    )
    file.write("ENABLE_CACHE=false\n")
    file.write("GITHUB_ACCESS_TOKEN={}\n".format(github_token))

print("Setup successfully!")

Setup successfully!


In [13]:
%%!
mycli -u $DB_PASSWORD -h $DB_ENDPOINT -P 4000 -D test --ssl-ca=/etc/ssl/certs/ca-certificates.crt --ssl-verify-server-cert

['(2003, "Can\'t connect to MySQL server on \'localhost\' ([Errno 111] Connection refused)")']

In [ ]:
%%!
curl -s \
  -H "Accept: application/vnd.github.v3+json" \
  -H "Authorization: token $GITHUB_TOKEN" \
  https://api.github.com/events